#### The Cell Class

In [1]:
# Class cell -> its the class of all the nodes in the grid, Each cell of the grid is defined as a object of type 
# cell.
class cell:
    def __init__(self,x,y): # Constructor for class cell
        self.x = x # x coordinate of the cell
        self.y = y # y coordinate of the cell
        self.g = 0 # g value of the cell (the distance from start cell to curr cell)
        self.h = 0 # h value of the cell (estimated value of the distance between curr cell and goal cell)
        self.f = 0 # f value of the cell (f = g+h in A*, defines the priority of a node)
        # Used in Inference Agent 4 and 5
        self.N_x = 0 # number of neighbors of cell x
        self.visited = False # whether the cell has been visited by the inference agent or not keep this info across all repeated A* calls
        self.C_x = -1 # number of neighbors sensed to be blocked
        self.B_x = 0 # number of neighbors confirmed to be blocked
        self.E_x = 0 # number of neighbors confirmed to be empty
        self.H_x = 0 # number of cells still hidden or unconfirmed
        self.blocked = False # Flag to indicate whether the cell is blocked or not
        self.neighbors_updated = False # used to prevent unnecessary information update in the agent
        # Used in Agent 5
        self.prob = 0 # Individual probability of a cell being blocked or not calculated using the formula C_x - B_x / H_x of the parent.
        self.min_Hx = 8 # The H_x value of parent which assigned the value of probability for this cell. 8 is the maximum value and hence is used. ]
        # Since a node has a lot of parents we want to assign it with probability which is the most certain i.e. from a parent whose H_x is minimum.
        self.min_Hx_ind = () # Index of the neighbour whose H_x, C_x and B_x values are used to assign the probabilities
    
    def set_h_value(self, g_x, g_y, heuristic, weight):
        '''This function is used to assign the heuristic value i.e. the h value and the f value to the calling 
           class object. It takes in the argument g_x = goal cell abscissa, g_y = goal cell ordinate, 
           heuristic = It defines the type of heuristic the A* problem is using, weight = this is for problem 
           number 9 where we test the effect of weights on the performance of repeated A* and normal A*.'''
        if heuristic == 'm':  # We have used manhattan heuristic in this project.
            # Manhattan distance
            self.h = weight * abs(g_x - self.x) + abs(g_y - self.y)
            self.f = self.g + self.h + self.prob # also updating f inside this function 
            # We are simply adding the probability of the blockage in the f value of the node.
        if heuristic == 'e':
            # euclidean distance
            self.h = weight * math.sqrt((g_x - self.x)**2 + (g_y - self.y)**2)
            self.f = self.g + self.h + self.prob
        if heuristic == 'c':
            # chebyshev distance
            self.h = weight * max(abs(g_x - self.x), abs(g_y - self.y))
            self.f = self.g + self.h + self.prob

#### Updated A* and repeated A*

In [2]:
def Astar(grid,start_cell_ind,goal_cell_ind, heuristic, weight = 1):
    cells_processed = 0 # number of cells popped from the fringe before reaching the goal.
    number_of_clashes = 0 # number of time we had to update priorities in the fringe.
    start_cell = grid[start_cell_ind[0]][start_cell_ind[1]] # start cell of our grid.
    goal_cell = grid[goal_cell_ind[0]][goal_cell_ind[1]] # goal cell of our grid
    open_list = SortedSet() # fringe, we have used sorted set as our priority queue
    close_list = set() # contains cells already processed, have used set for this purpose (O(1) lookup on average)
    parent_dict = {} # stores parent child relationship, using dictionary for this
    visited = set() # child already visited, using set for this purpose (O(1) lookup on average)
    start_cell.g = 0 # Setting the g value of the start cell to 0
    # setting the h and f value of the start cell using the weight and heuristic passed in the function arguments.
    # weight is for q9
    start_cell.set_h_value(goal_cell_ind[0], goal_cell_ind[1], heuristic, weight) 
    open_list.add((start_cell.f, start_cell.h, (start_cell.x, start_cell.y))) # adding the start cell to the priority queue
    # In the queue we are pushing a tuple whose first coordinate is the f value, second coordinate is the h value 
    # then a tuple containing the location of the cell. The priority is set according to the f value in case of a 
    # tie h value is seen.
    visited.add((start_cell.x, start_cell.y)) # updating the visited set as we are pushing the node in priority queue
    while len(open_list)>0: # loop over the priority queue until it is not empty or we reach the goal node
        cl = open_list.pop(0) # pop the node with the least priority (it is at index 0 in the sorted set)
        curr_cell = grid[cl[-1][0]][cl[-1][1]] # the last index in cl contains the x and y coordinates, 
        # storing the node popped from the queue in curr_cell.
        cells_processed += 1 # incrementing the number of cells processed (since we just popped from the queue)
        close_list.add((curr_cell.x, curr_cell.y)) # adding the node to the closed list as it is about to be processed
        if curr_cell == goal_cell: 
            # If the curr cell is the goal cell return the path from the curr cell to goal cell by backtracking 
            # from goal cell to curr cell using the parent dictionary as well as all the nodes 
            # that have been processed uptil now.
            path = []
            curr = curr_cell
            while (curr.x, curr.y) != (start_cell_ind[0], start_cell_ind[1]):
                path.append([curr.x, curr.y])
                curr = parent_dict[(curr.x, curr.y)]
            path.append([curr.x,curr.y])
            return path[::-1], cells_processed, number_of_clashes  
        children = [] # This list is used to store all the possible valid children of the cell 
        # currently being processed.
        for new_pos in [(0,1),(0,-1),(1,0),(-1,0)]: # 4 main compass directions, field of view also the agent can 
            # only move to these locations
            node_pos = (curr_cell.x+new_pos[0], curr_cell.y+new_pos[1]) # position of the child
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                # if the child is not part of the grid continue
                continue
            if(grid[node_pos[0]][node_pos[1]].blocked):
                # if the child is blocked continue
                continue
            if((node_pos[0], node_pos[1]) in close_list):
                # if child is in close list continue
                continue
            if((node_pos[0], node_pos[1]) in visited):
                # if the child is already visited then compare the previous cost(f) to the new cost(f) and if the
                # new cost is lower update the priority of the child in the priority queue, update the 
                # parent dictionary and update the g value of the node.
                if curr_cell.g + 1 + grid[node_pos[0]][node_pos[1]].h + grid[node_pos[0]][node_pos[1]].prob < grid[node_pos[0]][node_pos[1]].f:
                    number_of_clashes += 1 # increment the number of clashes by 1
                    # To update the priority we remove the node from the sorted set (O(logn)) and reinsert with 
                    # new priority O(log(n)). 
                    open_list.remove((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    grid[node_pos[0]][node_pos[1]].g = curr_cell.g+1
                    grid[node_pos[0]][node_pos[1]].set_h_value(goal_cell_ind[0], goal_cell_ind[1], heuristic, weight)
                    open_list.add((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    parent_dict[(node_pos[0],node_pos[1])] = curr_cell # updating the parent dictionary
            else:
                # If the child is not visited update its g value, set its h and f value and add it to the 
                # children list after updating its parent.
                grid[node_pos[0]][node_pos[1]].g = curr_cell.g + 1
                grid[node_pos[0]][node_pos[1]].set_h_value(goal_cell_ind[0], goal_cell_ind[1], heuristic, weight)
                children.append(grid[node_pos[0]][node_pos[1]])
                parent_dict[(node_pos[0],node_pos[1])] = curr_cell
        if(children == []): # If no valid child continue
            continue
        for child in children: # else if valid children exist add them to the open list and update the visited set.
            open_list.add((child.f,child.h, (child.x, child.y)))
            visited.add((child.x, child.y)) 
    return [], 0, 0

In [3]:
# function to implement the A* algorithm
def repeated_astar(bot_env, real_world_grid, start_cell_ind, goal_cell_ind, heuristic, final_discovered_grid_world, limited_view = False, weight=1, inf_bot = False):
    '''
    This function is the implementation of the repeated A* algorithm. This function calls A* algorithm whenver a 
    blocked cell is encountered along the path returned by A*.
    '''
    count = 0 # Number of A* calls needed to solve a grid by different agents
    bumps = 0 # Number of times a agent bumpped into an obstacle
    paths = [] # stores the path to be returned.
    cp = 0  # keeps count of the total number of cells processed in each A* call.
    kb = [] # This is an variable we used to check the amount of equations that were left in the knowledge base after the complete execution of agent 4.
    # Used to answer the question whether agent 4 refers everything or not there is to refer.
    planning_time = 0 # Used to calculate planning time of different agent, used in comparison of different agents.
    nodes_disc = 0 # Used to compare different inference agents, used to compare the amount of enviornment each agent discovered
    while(1):
        count += 1 # incrementing the amount of time A* was called
        t = time.time()
        path, cells_processed, n_c = Astar(bot_env, start_cell_ind, goal_cell_ind, heuristic, weight) # call A* with current knowledge of the grid
        planning_time += time.time() - t # Incrementing the planning time by the amount of time it took to run A* algo(planning a path)
        cp += cells_processed # add the number of cells processed to the variable cp which keeps the total number of cells processed for each iteration of A*
        if(path == []): # If A* doesn't return a path, path doesn't exist. Break from the loop and print "Maze is not solvable"
            print("Maze is not solveable")
            break
        else:  # If path exists, do this:
            if not limited_view: # If the limited view flag is off that means it is not the blind agent.
                if not inf_bot: # If it is not an inference agent i.e. It is agent 1 from Project 1
                    p = explore_path(path,bot_env,real_world_grid, final_discovered_grid_world) #explore path with complete field of view (Agent 1)
                else:
                    if inf_bot == 3: # If it is the example agent (I.e. Agent 3)
                        p, bumps = explore_path_new_agent(path,bot_env,real_world_grid, final_discovered_grid_world, bumps) # Call explore path of agent 3
                    elif inf_bot == 4: # If it is our agent 4 (The inference agent we made).
                        p, bumps,kb = explore_path_new_agent_4(path,bot_env,real_world_grid, final_discovered_grid_world, bumps, kb) # Call explore path for agent 4
                    else: # If it is agent 5.
                        p, bumps,kb = explore_path_new_agent_5(path,bot_env,real_world_grid, final_discovered_grid_world, bumps, kb) # Call explore path for agent 5
            else: # If the agent is blind
                p = explore_path_with_limited_view(path,bot_env,real_world_grid, final_discovered_grid_world) #explore path with limited view
            paths.append(p)  # append the path returned to paths  
            if p[-1] == goal_cell_ind: # if the last cell is goal cell, return paths, cells processed, number of bumps, planning time and node discovered in the enviorment (I.e. The number of nodes for which our agent has the complete information.)
                # Here we are calculating the number of nodes for which our agent has the complete information
                for i in range(len(final_discovered_grid_world)):
                    for j in range(len(final_discovered_grid_world[i])):
                        # If an node is unblocked in the final disc gridworld means that the agent discovered it to be unblocked as we assumed that everything is blocked in the final disc enviornement
                        if final_discovered_grid_world[i][j].blocked == False:
                            nodes_disc += 1 # Inc num of disc nodes
                for i in range(len(bot_env)):
                    for j in range(len(bot_env)):
                        # If an node is blocked in the bot_env means that the agent disc it to be blocked since we assume free space assumption for bot_env
                        if bot_env[i][j].blocked == True:
                            nodes_disc += 1 # Inc num of disc nodes
                return paths, cp, count, bumps, planning_time, nodes_disc, len(kb)
            else:    # if the last cell is not goal cell
                start_cell_ind = p[-1] # Restart repeated A* from the last cell returned by the explore path functions

#### Explore path Functions

###### explore_path (agent 1) and explore_path_with_limited_view (the blind agent) are from project 1. Hence not attaching their codes in this appendix.

#### Explore path function for agent 3

In [4]:
# function to explore the path returned by A* algorithm. Update the bot_env and final_discovered_grid_world 
# Also this is for the example inference agent (Agent 3)
def explore_path_new_agent(path,bot_env ,grid, final_discovered_grid_world, bumps):
    '''This functions helps move the agent along the path returned by A*. We update the bot enviornment and final 
    discovered grid world as the agent moves along the path. We also make inferences using the knowledge base rules 
    of the example agent.This function returns when the agent encounters a block along the path or infers if there is
    a block along the planned path or reaches the goal.'''  
    def update_neighbors(x, y, q, nodes_in_queue):
        '''This function is used to update the B_x, H_x and E_x value of the neighbours of a cell we just discovered.'''
        if(final_discovered_grid_world[x][y].neighbors_updated == False): # If the neighbours of the node for which we called 
            # this functions were already updated about the status of the cell do not reupdate the neighbours.
            for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: # looping through all the neighbours
                node_pos = (x+new_pos[0], y+new_pos[1])
                if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                    # If the neighbour is not valid continue
                    continue
                if(final_discovered_grid_world[x][y].blocked):
                    # If the node is blocked, Increment the B_x value for the neighbours and decrement the H_x value.
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].B_x += 1
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].H_x -= 1
                    bot_env[node_pos[0]][node_pos[1]].B_x += 1
                    bot_env[node_pos[0]][node_pos[1]].H_x -= 1
                else:
                    # If the node is unblocked, Increment the E_x value for the neighbours and decrement the H_x value.
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].E_x += 1
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].H_x -= 1
                    bot_env[node_pos[0]][node_pos[1]].E_x += 1
                    bot_env[node_pos[0]][node_pos[1]].H_x -= 1
                if((node_pos[0], node_pos[1]) not in nodes_in_queue): 
                # If the neighbour is already present in the queue don't add the neighbours to the queue again
                    q.put(node_pos)
                    nodes_in_queue.add((node_pos[0], node_pos[1]))
            final_discovered_grid_world[x][y].neighbors_updated = True # Mark the flag neighbours_updated to true
            bot_env[x][y].neighbors_updated = True
        return q, nodes_in_queue # Return updated queue and nodes_in_queue set
    # path is a list of list -> [[0,0], [0,2]......]
    for i in range(len(path)): # looping over the path list
        x = path[i][0] # x coordinate of the cell in the path 
        y = path[i][1] # y coordinate of the cell in the path
        # updating the information in the bot_env of the curr_cell
        bot_env[x][y].x = grid[x][y].x
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        # updating the information in the final discovered grid world of the curr_cell
        final_discovered_grid_world[x][y].x = grid[x][y].x
        final_discovered_grid_world[x][y].y = grid[x][y].y
        final_discovered_grid_world[x][y].blocked = grid[x][y].blocked
        # If visiting for the first time sense neighbors and mark visited
        num_of_child = 0 # used to calculate N_x value for the cell
        c_x = 0 # The number of sensed blocked cells in the neighborhood.
        if(final_discovered_grid_world[x][y].visited == False):
            # Looping over all the possible children in 8 directions.
            for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: 
                node_pos = (x+new_pos[0], y+new_pos[1])
                if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                    # If the child is not valid i.e. outside the grid continue
                    continue
                # If the child is valid increment the number of valid children and if the child is blocked 
                # increment the number of sensed blocked neighbours.
                num_of_child += 1 
                if(final_discovered_grid_world[x][y].blocked == False):
                    if(grid[node_pos[0]][node_pos[1]].blocked):
                        c_x += 1
            # Update N_x, C_x, H_x and mark visited for a cell in final discovered grid world and bot enviornment
            bot_env[x][y].N_x = num_of_child
            if(bot_env[x][y].blocked == False): # Do not update C_x value for the blocked cell.
                bot_env[x][y].C_x = c_x
            bot_env[x][y].visited = True
            bot_env[x][y].H_x += num_of_child
            final_discovered_grid_world[x][y].N_x = num_of_child
            if(final_discovered_grid_world[x][y].blocked == False): # Do not update the C_x value for a blocked cell.
                final_discovered_grid_world[x][y].C_x = c_x
            final_discovered_grid_world[x][y].visited = True
            final_discovered_grid_world[x][y].H_x += num_of_child
        # Queue to store all the nodes whose status we want to propogate to its neighbours and all the nodes whose value for B_x or E_x changed.
        q = queue.Queue() # normal queue
        q.put((x,y)) # put the node on he path in the queue as we just discovered it to be blocked or not and want to update it's neighbours B_x, E_x and H_x.
        nodes_in_queue = set() # We have used a set to do book keeping of the nodes in the queue as it helps in preventing readding a node to the queue which was already present.
        nodes_in_queue.add((x,y)) # O(1) time to add and find.
        
        if final_discovered_grid_world[x][y].C_x == -1:
            # If the node we encounted on the path is blocked, just update the neighbours B_x and H_x value.
            q, nodes_in_queue = update_neighbors(x,y,q, nodes_in_queue)
        # looping over the queue until we have updated all the neighbours of the cell in the queue and 
        # checked the neighbours against the knowledge base conditions
        while(not q.empty()):
            pos = q.get() # pop the element from the queue 
            nodes_in_queue.remove((pos[0], pos[1])) # also update the nodes_in_queue set
            if (final_discovered_grid_world[pos[0]][pos[1]].C_x == -1):
                # If the C_x value is -1 i.e. we have not visited this node continue as we cannot compare 
                # this node against the knowledge base as we don't know it's C_x yet. 
                # The cell might also be blocked in that case also we don't want to compare the cell with knowledge base 
                # conditions.
                continue
            q, nodes_in_queue = update_neighbors(pos[0],pos[1],q, nodes_in_queue) # Update the neighbours of the cell popped from the queue (update their B_x, E_x and H_x).
            # Call the knowledge base function to check if the current node satisfies any conditions in the knowledge base, this helps in making inference about other nodes of the grid.
            nodes_infered = knowledge_base(final_discovered_grid_world,bot_env,pos)
            # If we were not able to infer anything new continue to the next cell in the queue.
            if len(nodes_infered) == 0:
                continue
            else: # If we were able to infer something about a cell, update its neighbours and add the cell 
                # and it's neighbours to the queue if not already present. This is how we propogate information in agent 3 
                # till it is possible to infer something new.
                for node in nodes_infered:
                    if node in nodes_in_queue:
                        continue
                    else:
                        nodes_in_queue.add(node)
                        q.put(node)
                        q, nodes_in_queue = update_neighbors(node[0], node[1], q, nodes_in_queue)
        if(grid[x][y].blocked): # if we encounter the blocked cell return the path till the last unblocked cell
            bumps += 1 # Increment the number of bumps
            return (path[:i], bumps)
        else:
            # If the agent is able to infer a block cell along the path return the path till the current cell
            infered_block = False
            for j in range(i+1,len(path)):
                if bot_env[path[j][0]][path[j][1]].blocked:
                    infered_block = True
                    break             
            if(infered_block == True):
                return (path[:i+1], bumps)
    return (path, bumps)

#### Explore path function for agent 4

In [5]:
# kb should be an empty set during the first call out.
def explore_path_new_agent_4(path,bot_env ,grid, final_discovered_grid_world, bumps, kb):
    for i in range(len(path)): # looping over the path list
        # Mark in final discovered and bot_env if the cell is blocked or unblocked also mark visited.
        x = path[i][0] # x coordinate of the cell in the path 
        y = path[i][1] # y coordinate of the cell in the path
        bot_env[x][y].x = grid[x][y].x
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        final_discovered_grid_world[x][y].x = grid[x][y].x
        final_discovered_grid_world[x][y].y = grid[x][y].y
        final_discovered_grid_world[x][y].blocked = grid[x][y].blocked
        # If the cell was previously visited no need to generate its own equation and its neighbours equation.
        if(final_discovered_grid_world[x][y].visited == True):
            # already done previously and compared with knowledge base just continue.
            continue
        # Mark the nodes visited in the bot_env and final_disc_grid_world
        final_discovered_grid_world[x][y].visited = True
        bot_env[x][y].visited = True
        # Calculate the C_x for the cell. Used in calculating the RHS of the equations.
        c_x = 0
        eqn_neighbors = [set(),0] # equation of the neighbours
        for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: # looping over all potential childs
            node_pos = (x+new_pos[0], y+new_pos[1])
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                continue # If invalid child, continue
            eqn_neighbors[0].add((node_pos[0], node_pos[1])) # add the tuple of position of child to the lhs of the equation.
            if(grid[node_pos[0]][node_pos[1]].blocked):
                eqn_neighbors[1] += 1 # If the child is blocked increment the rhs of the equation.
                c_x += 1
        final_discovered_grid_world[x][y].C_x = c_x # Update the c_x value of the cell in the fdg and bot_env
        bot_env[x][y].C_x = c_x
        eqn = [set(),0] # Equation of the current node.
        eqn[0].add((x,y)) # adding the loc in tuple form in the lhs of the equation.
        if(grid[x][y].blocked): 
            eqn[1] += 1 # If the node is blocked increment the right hand side of the equation.
        q = deque() # We have used a double ended queue to store the equations we want to check against the knowledge base.
        q.append(eqn.copy()) # add the equation in deque. Since set is by reference in python we need to 
        # add the copy of the set to the queue as making changes throws error.
        
        if(eqn_neighbors[1] == 0): # If the rhs of neighbours eqn is 0 i.e. there are no sensed blocks in the neighborhood
        # The equation is solved and hence we need to assign 0 to each individual cell equation and add them to the dequeue.
            for tup in eqn_neighbors[0]: 
                t = [{tup}, 0]
                q.append(t.copy())
        # If the len(lhs) = rhs implies all the cells in the equation are blocked. The equation is solved and we 
        # assign 1 to each individual cell equation and add them to the dequeue.
        elif(len(eqn_neighbors[0]) == eqn_neighbors[1]):
            for tup in eqn_neighbors[0]:
                t = [{tup}, 1]
                q.append(t.copy())
        else:
            # The equation is not solved already, add it to the queue to check against the knowledge base.
            q.append(eqn_neighbors.copy())
        while(len(q)): # Iterating over all the equations in the queue.
            eqn_popped_from_q = q.popleft() # popping the equation from the queue.
            if(len(eqn_popped_from_q[0]) == 1): 
                # If the equation popped from the queue is of length 1 implies that either it is the equation of the 
                # node in which the agent is present in or the equation of a node the agent was able to infer about.
                # In such cases we update the final discovered gridworld and the bot_env about the staus of the cell 
                # and then process the knowledge base for reductions using the subsequence and set_diff method.
                ind = next(iter(eqn_popped_from_q[0]))
                if(eqn_popped_from_q[1] == 0):
                    final_discovered_grid_world[ind[0]][ind[1]].blocked = False
                else:
                    bot_env[ind[0]][ind[1]].blocked = True
            else:
                # If the equation popped from the knowledge base is not of length one, first we reduce the equation
                # by removing the nodes about whome we have complete information and check if the reduced equation is 
                # not already solved. If it is already solved we add the individual cell equations to the queue otherwise
                # process this equation against the knowledge base.
                temp_eqn = eqn_popped_from_q[0].copy()
                for node_pos in eqn_popped_from_q[0]:
                    if(final_discovered_grid_world[node_pos[0]][node_pos[1]].blocked == False): 
                        temp_eqn.remove(node_pos)
                    elif(bot_env[node_pos[0]][node_pos[1]].blocked == True):
                        temp_eqn.remove(node_pos)
                        eqn_popped_from_q[1] -= 1
                eqn_popped_from_q[0] = temp_eqn
                if(len(eqn_popped_from_q[0]) == 0):
                    continue
                if(eqn_popped_from_q[1] == 0):
                    for tup in eqn_popped_from_q[0]:
                        tup1 = [{tup}, 0]
                        q.appendleft(tup1)
                    continue
                elif(eqn_popped_from_q[1] == len(eqn_popped_from_q[0])):
                    for tup in eqn_popped_from_q[0]:
                        tup1 = [{tup}, 1]
                        q.appendleft(tup1)
                    continue 
            # If the equation is already present in the kb do not reprocess the equation.
            if(eqn_popped_from_q in kb):
                continue
            # Call the subsequence and set_difference methods.
            q, sub_seq_flag = knowledge_base_agent_4_subsequence(final_discovered_grid_world, bot_env, eqn_popped_from_q, kb, q)
            q, set_diff_flag = knowledge_base_agent_4_set_diff(final_discovered_grid_world, bot_env, eqn_popped_from_q, kb, q)
            
            # If the condition (ii.) of subsequence method is true or the condition in set_difference method is satisfied
            # Do not store the equation popped from the queue to the knowledge base as we already have the representative
            # information about the equation in the knowledge base and the queue.
            if(sub_seq_flag or set_diff_flag):
                continue
            else:
                # Srore the equation in the knowledge base if it is not length one. (No need to store length one equations.)
                # Already marked in the final disc gridworld and bot_env.
                if(len(eqn_popped_from_q[0]) == 1):
                    continue
                kb.append(eqn_popped_from_q)
        if(grid[x][y].blocked): # If the current cell is blocked inc the number of bumps and return the path till the last unblocked index.
            bumps += 1
            return (path[:i], bumps, kb)
        else:
            # IF the agent is able to infer block along the planned path stop going forward and return the path till the current index.
            infered_block = False
            for j in range(i+1,len(path)):
                if bot_env[path[j][0]][path[j][1]].blocked:
                    infered_block = True
                    break
            if(infered_block == True):
                return (path[:i+1], bumps, kb)
    return (path, bumps, kb)

#### Explore path function for agent 5

In [6]:
def explore_path_new_agent_5(path,bot_env ,grid, final_discovered_grid_world, bumps, kb):
    def update_neighbors(x, y):
        '''This function updates the E_x, B_x and H_x values of the neighbours of the cell (x,y). It also 
        updates the probabilities of the neighbours of the cell that are already visited if updation is possible.'''
        if(final_discovered_grid_world[x][y].neighbors_updated == False):
            # If we have already updated neighbours once don't do again.
            for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: # loop over all possible children
                node_pos = (x+new_pos[0], y+new_pos[1])
                if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                    # If invalid children continue
                    continue
                if(final_discovered_grid_world[x][y].blocked):
                    # If the cell is blocked, increment B_x in neighbours and reduce H_x.
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].B_x += 1
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].H_x -= 1
                    bot_env[node_pos[0]][node_pos[1]].B_x += 1
                    bot_env[node_pos[0]][node_pos[1]].H_x -= 1
                    # If the neighbour is already visited by the agent, since it's H_x is changing we 
                    # need to update the probabilities of the neighbours of this neighbour.
                    if(final_discovered_grid_world[node_pos[0]][node_pos[1]].visited):
                        update_neighbors_prob(node_pos[0], node_pos[1]) # calling update neighbour probability function
                else:
                    # If the cell is unblocked, increment E_x in neighbours and reduce H_x.
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].E_x += 1
                    final_discovered_grid_world[node_pos[0]][node_pos[1]].H_x -= 1
                    bot_env[node_pos[0]][node_pos[1]].E_x += 1
                    bot_env[node_pos[0]][node_pos[1]].H_x -= 1
                    # If the neighbour is already visited by the agent, since it's H_x is changing we 
                    # need to update the probabilities of the neighbours of this neighbour.
                    if(final_discovered_grid_world[node_pos[0]][node_pos[1]].visited):
                        update_neighbors_prob(node_pos[0], node_pos[1])
            # Mark true the neighbours updated flag for (x,y).
            final_discovered_grid_world[x][y].neighbors_updated = True
            bot_env[x][y].neighbors_updated = True   
    def update_neighbors_prob(x,y):
        '''This function assigns probability to the neighbours of (x,y).'''
        for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: # looping through all the neighbours
            node_pos = (x+new_pos[0], y+new_pos[1])
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                # If it is an invalid neighbour continue.
                continue
            if(final_discovered_grid_world[node_pos[0]][node_pos[1]].blocked == False or bot_env[node_pos[0]][node_pos[1]].blocked == True):
                # If we know the status of a cell don't need to assign probability to that cell.
                continue
            # We compare the H_x value of the current node with H_x value of the node which assigned probability 
            # to the node_pos neighbour.
            new_hx = final_discovered_grid_world[x][y].H_x
            old_hx = final_discovered_grid_world[node_pos[0]][node_pos[1]].min_Hx
            # If the value of H_x for the current cell is less than that of h_x of the cell which assigned probability to node_pos
            # Update the probability of node_pos with the value C_x - B_x/ H_x of the current cell.
            if new_hx < old_hx:
                c_x = final_discovered_grid_world[x][y].C_x
                b_x = final_discovered_grid_world[x][y].B_x
                # Update min_hx, min_hx_ind and probability for node_pos.
                final_discovered_grid_world[node_pos[0]][node_pos[1]].min_Hx = new_hx
                final_discovered_grid_world[node_pos[0]][node_pos[1]].prob = (c_x - b_x)/new_hx
                final_discovered_grid_world[node_pos[0]][node_pos[1]].min_Hx_ind = (x,y)
                bot_env[node_pos[0]][node_pos[1]].min_Hx = new_hx
                bot_env[node_pos[0]][node_pos[1]].prob = (c_x - b_x)/new_hx
                bot_env[node_pos[0]][node_pos[1]].min_Hx_ind = (x,y)
            else:
                # If this is not the case continue.
                continue  
    for i in range(len(path)):
        x = path[i][0]
        y = path[i][1]
        bot_env[x][y].x = grid[x][y].x
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        final_discovered_grid_world[x][y].x = grid[x][y].x
        final_discovered_grid_world[x][y].y = grid[x][y].y
        final_discovered_grid_world[x][y].blocked = grid[x][y].blocked
        if(final_discovered_grid_world[x][y].visited == True):
            continue
        # Call the update neighbours function to update the B_x, E_x and H_x of the neighbours, also if a neighbour is already 
        # visited since its H_x value is changing we recheck the probabilities of the neighbours of this neighbour in this function.
        update_neighbors(x,y)
        final_discovered_grid_world[x][y].visited = True
        bot_env[x][y].visited = True
        c_x = 0
        num_of_childs = 0
        

# -------------------------------------------------------------------------------     
        # The code is same as agent 4 only adding comments to new piece of code.
# -------------------------------------------------------------------------------
        
        eqn_neighbors = [set(),0]
        for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: 
            node_pos = (x+new_pos[0], y+new_pos[1])
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                continue
            num_of_childs += 1
            eqn_neighbors[0].add((node_pos[0], node_pos[1]))
            if(grid[node_pos[0]][node_pos[1]].blocked):
                eqn_neighbors[1] += 1
                c_x += 1
        final_discovered_grid_world[x][y].C_x = c_x
        bot_env[x][y].C_x = c_x
        final_discovered_grid_world[x][y].N_x = num_of_childs
        final_discovered_grid_world[x][y].H_x += num_of_childs
        if (final_discovered_grid_world[x][y].blocked == True):
            final_discovered_grid_world[x][y].prob = 1
            bot_env[x][y].prob = 1
            update_neighbors_prob(x,y)
        else:
            final_discovered_grid_world[x][y].prob = 0
            bot_env[x][y].prob = 0
            update_neighbors_prob(x,y)
        eqn = [set(),0]
        eqn[0].add((x,y))
        if(grid[x][y].blocked):
            eqn[1] += 1
        q = deque()
        q.append(eqn.copy())
        if(eqn_neighbors[1] == 0):
            for tup in eqn_neighbors[0]:
                t = [{tup}, 0]
                q.append(t.copy())
        elif(len(eqn_neighbors[0]) == eqn_neighbors[1]):
            for tup in eqn_neighbors[0]:
                t = [{tup}, 1]
                q.append(t.copy())
        else:
            q.append(eqn_neighbors.copy())
        while(len(q)):
            eqn_popped_from_q = q.popleft()
            if(len(eqn_popped_from_q[0]) == 1):
                ind = next(iter(eqn_popped_from_q[0]))
                if(eqn_popped_from_q[1] == 0):
                    final_discovered_grid_world[ind[0]][ind[1]].blocked = False
                    final_discovered_grid_world[ind[0]][ind[1]].prob = 0
                    bot_env[ind[0]][ind[1]].prob = 0 # assigning 0 probability to unblocked cells
                    update_neighbors(ind[0], ind[1]) # Calling update neighbours function for the inferred cells.
                else:
                    bot_env[ind[0]][ind[1]].blocked = True
                    final_discovered_grid_world[ind[0]][ind[1]].prob = 1 # assigning 1 probability to blocked cells.
                    bot_env[ind[0]][ind[1]].prob = 1 # Calling update neighbours function for inferred cells.
                    update_neighbors(ind[0], ind[1])        
            else: 
                temp_eqn = eqn_popped_from_q[0].copy()
                for node_pos in eqn_popped_from_q[0]:
                    if(final_discovered_grid_world[node_pos[0]][node_pos[1]].blocked == False): 
                        temp_eqn.remove(node_pos)
                    elif(bot_env[node_pos[0]][node_pos[1]].blocked == True):
                        temp_eqn.remove(node_pos)
                        eqn_popped_from_q[1] -= 1
                eqn_popped_from_q[0] = temp_eqn
                if(len(eqn_popped_from_q[0]) == 0):
                    continue
                if(eqn_popped_from_q[1] == 0):
                    for tup in eqn_popped_from_q[0]:
                        tup1 = [{tup}, 0]
                        q.appendleft(tup1)
                    continue
                elif(eqn_popped_from_q[1] == len(eqn_popped_from_q[0])):
                    for tup in eqn_popped_from_q[0]:
                        tup1 = [{tup}, 1]
                        q.appendleft(tup1)
                    continue 
            if(eqn_popped_from_q in kb):
                continue
            q, sub_seq_flag = knowledge_base_agent_4_subsequence(final_discovered_grid_world, bot_env, eqn_popped_from_q, kb, q)
            q, set_diff_flag = knowledge_base_agent_4_set_diff(final_discovered_grid_world, bot_env, eqn_popped_from_q, kb, q)
            if(sub_seq_flag or set_diff_flag):
                continue
            else:
                if(len(eqn_popped_from_q[0]) == 1):
                    continue
                kb.append(eqn_popped_from_q)
        if(grid[x][y].blocked):
            bumps += 1
            return (path[:i], bumps, kb)
        else:
            infered_block = False
            for j in range(i+1,len(path)):
                if bot_env[path[j][0]][path[j][1]].blocked:
                    infered_block = True
                    break
                        
            if(infered_block == True):
                return (path[:i+1], bumps, kb)
    return (path, bumps, kb)

#### Knowledge base functions

#### Agent 3

In [7]:
def knowledge_base(final_discovered_grid_world, bot_env, node_pos):
    '''This function checks if for a node given it's B_x, C_x, N_x, H_x satisfies 3 conditions in the knoeledge 
    base: (1) if H_x = 0 -> nothing new to infer, (2) Cx = Bx all remaing neighbours of the node are unblocked
    (3) if Nx - Bx = Ex all remaining neighbours of the node are blocked.'''
    list_of_nodes_infered = set() # This set stores all the nodes we were able to infer using the knowledge base confitions.
    x = node_pos[0] # x coordinate of the cell in the grid
    y = node_pos[1] # y coordinate of the cell in the grid
    if (final_discovered_grid_world[x][y].H_x == 0): # If the H_x value of the node is 0, there is nothing to infer in the surrounding of this cell so return.
        return list_of_nodes_infered
    if (final_discovered_grid_world[x][y].C_x == final_discovered_grid_world[x][y].B_x):
        # This is condition (2) where Cx = Bx implies all the unvisited nodes are unblocked.
        for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]: # loop through all the neighbours.
            node_pos = (x+new_pos[0], y+new_pos[1])
            if(node_pos[0] > len(bot_env)-1 or node_pos[1] > len(bot_env)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                # If invalid neighbour continue
                continue
            else:
                if(bot_env[node_pos[0]][node_pos[1]].blocked):
                    continue
                if(final_discovered_grid_world[node_pos[0]][node_pos[1]].visited == True):
                    continue
                # The previous two conditions check if the node is already visited or not.
                if(final_discovered_grid_world[node_pos[0]][node_pos[1]].neighbors_updated == True):
                    # This condition checks if the neighbour has been previously infered, if it is no need to mark it as unblocked since we already know it's value.
                    # We check this by checking the value of nodes_infered flag since we update neighbours when we infer a node, in the explore path function.
                    continue
                # If the cell is nither visited nor it was infered to be blocked or unblocked previously, we mark it as unblocked.
                final_discovered_grid_world[node_pos[0]][node_pos[1]].blocked = False
                list_of_nodes_infered.add((node_pos[0], node_pos[1])) # add the neighbour to the nodes_infered list.
        return list_of_nodes_infered
    if (final_discovered_grid_world[x][y].N_x - final_discovered_grid_world[x][y].C_x == final_discovered_grid_world[x][y].E_x):
        # This is condition (3) where Nx - Cx = Bx. Comments are same as condition (2), nothing new.
        for new_pos in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]:
            node_pos = (x+new_pos[0], y+new_pos[1])
            if(node_pos[0] > len(bot_env)-1 or node_pos[1] > len(bot_env)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                continue
            else:
                if(final_discovered_grid_world[node_pos[0]][node_pos[1]].blocked == False):
                    continue
                if(bot_env[node_pos[0]][node_pos[1]].visited == True):
                    continue
                if(bot_env[node_pos[0]][node_pos[1]].neighbors_updated == True):
                    continue
                bot_env[node_pos[0]][node_pos[1]].blocked = True
                list_of_nodes_infered.add((node_pos[0], node_pos[1]))
        return list_of_nodes_infered
    return list_of_nodes_infered

#### Agent 4 and 5 use the same knowledge base functions

##### The subsequence method

In [8]:
def knowledge_base_agent_4_subsequence(final_discovered_grid_world, bot_env, eqn, kb, q):
    '''This is the function for the subsequence method described in the report.'''
    sub_seq_flag = False # The subsequence flag, we mark this true when we find an equation in the knowledge base which is the subsequence of 
    # the equation popped from the queue => don't store the popped equation in knowledge base (redundant).
    for kb_eqn in kb: # looping over all the equations in the knowledge base.
        if (len(kb_eqn[0]) >= len(eqn[0])): # if the length of equation in the knowledge base is greater than or equal to the length of equation popped from the queue
            if (eqn[0].issubset(kb_eqn[0]) == True): # Check if the equation popped from the queue is the subsequence of equation in the knowledge base
                kb.remove(kb_eqn) # Remove the knowledge base equation
                kb_eqn[0] = kb_eqn[0].difference(eqn[0]) # reduce the knowledge base equation by removing the equation popped from the queue from the knowledge base equation
                kb_eqn[1] = kb_eqn[1] - eqn[1] # reduce the rhs of knowledge base equation
                # Check for the solvebility of the updated knowledge base equation
                if(len(kb_eqn[0]) == kb_eqn[1]): # If the number of variables in the equation = rhs => all variables have value 1.
                    # i.e. all cells are blocked
                    for i in kb_eqn[0]: # appending to the front of the dequeue the individual cell equations.
                        t = [{i}, 1] # assigning each cell the value of 1
                        q.appendleft(t) # append left the cell equation to the dequeue
                elif(kb_eqn[1] == 0): # If the rhs of the equation is 0 => all the variables have value 0.
                    for i in kb_eqn[0]: # appending to the front of the dequeue the individual cell equations.
                        t = [{i}, 0] # assigning each cell a value of 0.
                        q.appendleft(t) # append left the cell equation to the dequeue.
                else:
                    q.append(kb_eqn) # If the reduced equation is not solvable simply append it to the back of dequeue.
        else:
            # if the length of equation popped from the queue is more than the length of equation in the knowledge base
            temp = eqn # storing the eqn in temp
            if (kb_eqn[0].issubset(temp[0]) == True): # If the kb eqn is subsequence of equation popped from the queue
                temp[0] = temp[0].difference(kb_eqn[0]) # subtract the knowledge base equation from the equation popped from the queue
                temp[1] = temp[1] - kb_eqn[1] # subtract the rhs
                sub_seq_flag = True # Set the sub_seq_flag to true as we found a equation in knowledge base which is the subsequence of equation
                # popped from the queue
                if(len(temp[0]) == temp[1]): # If the reduced equation popped from the queue is solvable append individual cell equation to the front of dequeue
                    for i in temp[0]:
                        t = [{i}, 1]
                        q.appendleft(t)
                elif(len(temp[0]) == 0):
                    for i in temp[0]:
                        t = [{i}, 0]
                        q.appendleft(t)
                else:
                    q.appendleft(temp) # Else if not solvable simply append the reduced equation to the back of dequeue
    return q, sub_seq_flag     

##### The Set Difference method

In [9]:
def knowledge_base_agent_4_set_diff(final_discovered_grid_world, bot_env, eqn, kb, q):
    '''This is the function for set difference method described in the report.'''
    set_diff_flag = False # We mark this true when set diff of equation popped from the queue and kb equation 
    # is solvable i.e. A-B or B-A is solvable => we dont need to store the equation popped from the queue to the knowledge base.
    for kb_eqn in kb: # looping over all the equations in the knowledge base.
        if kb_eqn[1] >= eqn[1]: # if the RHS of knowledge base equation is more than the RHS of equation popped from the queue
            # Do kb_eqn - eqn
            sd = kb_eqn[0].difference(eqn[0]) # This is the set difference of kb_eqn - eqn.
            if(len(sd) == 0): # if both eqn's same continue (This never happens but just in case)
                continue
            if(len(sd) == kb_eqn[1]-eqn[1]): # If the equation is solvable
                set_diff_flag = True # mark the flag true and append the individual cell equations to the front of the dequeue
                for i in sd:
                    t = [{i}, 1]
                    q.appendleft(t)
                for i in eqn[0].difference(kb_eqn[0]):
                    t = [{i}, 0]
                    q.appendleft(t)
        else:
            # If the RHS of the equation popped from the queue is greater than the RHS of the equation in the knowledge base
            sd = eqn[0].difference(kb_eqn[0]) # Do eqn - kb_eqn
            if(len(sd) == 0):
                continue
            if(len(sd) == eqn[1] - kb_eqn[1]): # IF THIS IS SOLVABLE
                set_diff_flag = True # mark the flag as true and append the individual cell equations to the front of the deueue
                for i in sd: 
                    t = [{i}, 1]
                    q.appendleft(t)
                for i in kb_eqn[0].difference(eqn[0]):
                    t = [{i}, 0]
                    q.appendleft(t)
                    
    return q, set_diff_flag